In [3]:
!pip install pillow

import numpy as np

import tensorflow as tf
from tensorflow import keras

In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [5]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-28 13:37:03.715656: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-28 13:37:03.717131: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
img = load_img('1280px-Smaug_par_David_Demaret.jpg', target_size=(150, 150))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

preds = model.predict(X)

2022-11-28 13:37:58.575195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 637ms/step


In [8]:
preds

array([[1.]], dtype=float32)

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dino_dragon.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/wg/jtm_b31d60jgtpd71c8mfj2h0000gn/T/tmp9zp79ys5/assets


INFO:tensorflow:Assets written to: /var/folders/wg/jtm_b31d60jgtpd71c8mfj2h0000gn/T/tmp9zp79ys5/assets
2022-11-28 16:24:45.320409: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-28 16:24:45.320424: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-28 16:24:45.322049: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/wg/jtm_b31d60jgtpd71c8mfj2h0000gn/T/tmp9zp79ys5
2022-11-28 16:24:45.323500: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-28 16:24:45.323506: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/wg/jtm_b31d60jgtpd71c8mfj2h0000gn/T/tmp9zp79ys5
2022-11-28 16:24:45.325877: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-28 16:24:45.326508: I tensorflow/cc/saved_model/load

### Question 1

In [12]:
!ls -lh dino_dragon.tflite

-rw-r--r--  1 daniel  staff    43M Nov 28 16:24 dino_dragon.tflite


### Question 2

In [11]:
import tensorflow.lite as tflite

In [12]:
interpreter = tflite.Interpreter(model_path='dino_dragon.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

output_index

13

In [13]:
target_size = tuple(interpreter.get_input_details()[0]['shape'][1:3])
target_size

(150, 150)

In [14]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

### Question 3

In [16]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
image_file = download_image(url=url)
img = prepare_image(img=image_file, target_size=target_size)

/var/folders/wg/jtm_b31d60jgtpd71c8mfj2h0000gn/T/ipykernel_70507/2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


In [19]:
x = np.array(img)
X = np.array([x / 255], dtype=np.float32)
X

array([[[[0.5529412 , 0.31764707, 0.1764706 ],
         [0.57254905, 0.34117648, 0.1764706 ],
         [0.5647059 , 0.3254902 , 0.14901961],
         ...,
         [0.07058824, 0.02745098, 0.01960784],
         [0.07058824, 0.03921569, 0.02745098],
         [0.10980392, 0.04705882, 0.04705882]],

        [[0.5137255 , 0.28235295, 0.14901961],
         [0.5529412 , 0.30980393, 0.15686275],
         [0.5803922 , 0.34901962, 0.18431373],
         ...,
         [0.05882353, 0.04313726, 0.03137255],
         [0.05490196, 0.02352941, 0.01176471],
         [0.09803922, 0.03529412, 0.03921569]],

        [[0.5058824 , 0.29411766, 0.15294118],
         [0.5686275 , 0.34117648, 0.19607843],
         [0.5568628 , 0.32156864, 0.1764706 ],
         ...,
         [0.07450981, 0.02745098, 0.02745098],
         [0.09019608, 0.03529412, 0.03137255],
         [0.07450981, 0.03529412, 0.03137255]],

        ...,

        [[0.29803923, 0.41960785, 0.36078432],
         [0.4       , 0.36078432, 0.3529412 ]

Answer is 0.5529412

### Question 4

In [20]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

interpreter.get_tensor(output_index)[0][0]

0.82448643

In [6]:
!docker pull svizor42/zoomcamp-dino-dragon-lambda:v2

v2: Pulling from svizor42/zoomcamp-dino-dragon-lambda

8a52bef6: Already exists 
8f2d3828: Already exists 
b5cc6248: Already exists 
0148b8a0: Already exists 
b7dafa1a: Already exists 
bb8c0ba3: Already exists 
98799db2: Already exists Digest: sha256:46be01f7953cd48ebe526329ddf781d92019b9d44de4ecb67d11f56eaccf79cb
Status: Downloaded newer image for svizor42/zoomcamp-dino-dragon-lambda:v2
docker.io/svizor42/zoomcamp-dino-dragon-lambda:v2


### Question 5

In [9]:
!docker images svizor42/zoomcamp-dino-dragon-lambda

REPOSITORY                             TAG       IMAGE ID       CREATED       SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2        20ef58b21a05   12 days ago   639MB


In [10]:
!docker build --platform linux/amd64 -t dino_dragon-model .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/svizor42/zoomcamp-dino-dragon-  0.0s
[+] Building 0.1s (9/9) FINISHED                                                
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

### Question 6

In [11]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'}

result = requests.post(url, json=data).json()
print(result)

{'prediction': 0.31950676441192627}
